- wrangling & consistency check customer data
- combining ords_prods_merge with customer data
- exporing new data set as pkl

In [1]:
# Import Libraries
import pandas as pd
import numpy as np
import os

In [2]:
path = r'C:\Users\rieke\Data Analytics\Achievment 4\02-2023_Instacart Basket Analysis'

In [3]:
path

'C:\\Users\\rieke\\Data Analytics\\Achievment 4\\02-2023_Instacart Basket Analysis'

In [4]:
customers = pd.read_csv(os.path.join(path, '02 Data', 'Original Data', 'customers.csv'))

### Data wrangling & consistency

In [5]:
customers.head()

,user_id,First Name,Surnam,Gender,STATE,Age,date_joined,n_dependants,fam_status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374


In [6]:
# dropping names for privacy reason
customers = customers.drop(columns = ['First Name', 'Surnam'])

In [7]:
customers.head()

,user_id,Gender,STATE,Age,date_joined,n_dependants,fam_status,income
0,26711,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Female,Maryland,26,1/1/2017,1,married,40374


In [8]:
# looking for mixed type columns
for col in customers.columns.tolist():
  weird = (customers[[col]].applymap(type) != customers[[col]].iloc[0].apply(type)).any(axis = 1)
  if len (customers[weird]) > 0:
    print (col)

In [9]:
# rename non-uniform columns
customers.rename(columns = {'Gender':'gender', 'STATE':'state', 'Age':'age', 'n_dependants':'pers_p_household'}, inplace = True)

In [10]:
customers.head()

,user_id,gender,state,age,date_joined,pers_p_household,fam_status,income
0,26711,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Female,Maryland,26,1/1/2017,1,married,40374


In [11]:
customers.shape

(206209, 8)

In [12]:
customers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 206209 entries, 0 to 206208
Data columns (total 8 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   user_id           206209 non-null  int64 
 1   gender            206209 non-null  object
 2   state             206209 non-null  object
 3   age               206209 non-null  int64 
 4   date_joined       206209 non-null  object
 5   pers_p_household  206209 non-null  int64 
 6   fam_status        206209 non-null  object
 7   income            206209 non-null  int64 
dtypes: int64(4), object(4)
memory usage: 12.6+ MB


In [13]:
# change format to 1) avoid space issues and 2) further merging steps based on user_id 
customers['user_id'] = customers['user_id'].astype('int32')
customers['age'] = customers['age'].astype('int8')
customers['pers_p_household'] = customers['pers_p_household'].astype('int8')
customers['income'] = customers['income'].astype('int32')

In [14]:
customers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 206209 entries, 0 to 206208
Data columns (total 8 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   user_id           206209 non-null  int32 
 1   gender            206209 non-null  object
 2   state             206209 non-null  object
 3   age               206209 non-null  int8  
 4   date_joined       206209 non-null  object
 5   pers_p_household  206209 non-null  int8  
 6   fam_status        206209 non-null  object
 7   income            206209 non-null  int32 
dtypes: int32(2), int8(2), object(4)
memory usage: 8.3+ MB


In [15]:
customers.isnull().sum()

user_id             0
gender              0
state               0
age                 0
date_joined         0
pers_p_household    0
fam_status          0
income              0
dtype: int64

### combining data sets

In [16]:
ords_prods_merge = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'ords_prods_merge_wo_inf.pkl'))

In [17]:
ords_prods_merge.shape

(32399732, 23)

In [18]:
# merge data sets
ords_prods_cust = ords_prods_merge.merge(customers, on = ['user_id'], indicator = 'matches')

In [19]:
# run check 
ords_prods_cust['matches'].value_counts()

both          32399732
left_only            0
right_only           0
Name: matches, dtype: int64

In [21]:
ords_prods_cust.head(15)

,order_id,user_id,order_number,orders_day_of_week,order_time_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,...,order_frequency,cust_type,gender,state,age,date_joined,pers_p_household,fam_status,income,matches
0,2539329,1,1,2,8,NaN,196,1,0,Soda,...,20.5,Non-frequent customer,Female,Alabama,31,2/17/2019,3,married,40423,both
1,2398795,1,2,3,7,15.0,196,1,1,Soda,...,20.5,Non-frequent customer,Female,Alabama,31,2/17/2019,3,married,40423,both
2,473747,1,3,3,12,21.0,196,1,1,Soda,...,20.5,Non-frequent customer,Female,Alabama,31,2/17/2019,3,married,40423,both
3,2254736,1,4,4,7,29.0,196,1,1,Soda,...,20.5,Non-frequent customer,Female,Alabama,31,2/17/2019,3,married,40423,both
4,431534,1,5,4,15,28.0,196,1,1,Soda,...,20.5,Non-frequent customer,Female,Alabama,31,2/17/2019,3,married,40423,both
5,3367565,1,6,2,7,19.0,196,1,1,Soda,...,20.5,Non-frequent customer,Female,Alabama,31,2/17/2019,3,married,40423,both
6,550135,1,7,1,9,20.0,196,1,1,Soda,...,20.5,Non-frequent customer,Female,Alabama,31,2/17/2019,3,married,40423,both
7,3108588,1,8,1,14,14.0,196,2,1,Soda,...,20.5,Non-frequent customer,Female,Alabama,31,2/17/2019,3,married,40423,both
8,2295261,1,9,1,16,0.0,196,4,1,Soda,...,20.5,Non-frequent customer,Female,Alabama,31,2/17/2019,3,married,40423,both
9,2550362,1,10,4,8,30.0,196,1,1,Soda,...,20.5,Non-frequent customer,Female,Alabama,31,2/17/2019,3,married,40423,both


In [22]:
ords_prods_cust = ords_prods_cust.to_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'ords_prods_cust_merge.pkl'))